In [6]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import torch
import os
from torch import nn
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
import seaborn as sns
from ray import tune
import torch.optim as optim
from sklearn.metrics import accuracy_score
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

In [7]:
# import data from processed_data.csv for full set of small_set_processed_data.csv for3 city sample
df = pd.read_csv('small_set_processed_data.csv')
# df = pd.read_csv('processed_data.csv')

# split into X and y
X = df.drop('condition', axis=1)
y = df['condition']

# convert from boolean to int
y = y.astype(int)
X = X.astype(float)

# split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# convert to torch tensors
X_train = torch.tensor(X_train.values)
X_test = torch.tensor(X_test.values)
y_train = torch.tensor(y_train.values)
y_test = torch.tensor(y_test.values)

In [10]:
def train_model(config, checkpoint_dir=None, data_dir=None):
    # Define the neural network architecture
    net = nn.Sequential(
        nn.Linear(X_train.shape[1], config["hidden_size"]),
        nn.ReLU(),
        nn.Linear(config["hidden_size"], 2)
    )

    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=config["lr"])

    # Restore from checkpoint, if available
    if checkpoint_dir:
        checkpoint = torch.load(os.path.join(checkpoint_dir, "checkpoint"))
        net.load_state_dict(checkpoint["net"])
        optimizer.load_state_dict(checkpoint["optimizer"])
        epoch = checkpoint["epoch"]
        loss = checkpoint["loss"]
    else:
        epoch = 0
        loss = None

    # Train the model
    while epoch < config["num_epochs"]:
        optimizer.zero_grad()
        outputs = net(X_train.float())
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

        # Save checkpoint at the end of each epoch
        if checkpoint_dir:
            checkpoint_path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save({
                "net": net.state_dict(),
                "optimizer": optimizer.state_dict(),
                "epoch": epoch,
                "loss": loss
            }, checkpoint_path)

        epoch += 1

        # Report metrics at the end of each epoch
        net.eval()
        y_pred = net(X_test.float()).argmax(dim=1)
        accuracy = accuracy_score(y_test, y_pred)
        tune.report(mean_accuracy=accuracy, training_iteration=epoch)
    
    return {"mean_accuracy": accuracy}



In [13]:
config = {
    "lr": tune.loguniform(1e-4, 1e-1),
    "hidden_size": tune.choice([250, 500, 750, 1000]),
    "num_epochs": 10,
}

scheduler = ASHAScheduler(
    metric="mean_accuracy",
    mode="max",
    max_t=10,
    grace_period=1,
    reduction_factor=2
)

reporter = CLIReporter(metric_columns=["mean_accuracy", "training_iteration"])

analysis = tune.run(
    train_model,
    config=config,
    num_samples=10,
    scheduler=scheduler,
    progress_reporter=reporter,
)


print(analysis.best_config)
print(analysis.best_checkpoint)

2023-05-08 10:33:48,903	WARNING worker.py:1986 -- Warning: The actor ImplicitFunc is very large (78 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
2023-05-08 10:33:50,017	WARNING util.py:244 -- The `start_trial` operation took 3.687 s, which may be a performance bottleneck.


== Status ==
Current time: 2023-05-08 10:33:50 (running for 00:00:28.80)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Logical resource usage: 1.0/4 CPUs, 0/0 GPUs
Result logdir: /Users/alexisfrankson/ray_results/train_model_2023-05-08_10-33-21
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+-------------------------+----------+-----------------+---------------+-------------+
| Trial name              | status   | loc             |   hidden_size |          lr |
|-------------------------+----------+-----------------+---------------+-------------|
| train_model_480b9_00000 | RUNNING  | 127.0.0.1:27716 |           500 | 0.0073379   |
| train_model_480b9_00001 | PENDING  |                 |           500 | 0.00107933  |
| train_model_480b9_00002 | PENDING  |                 |           250 | 0.000305067 |
| train_model_480b9_00003 | PENDING  |                 |           250 | 0.00143448  |
| train_model_480b9_00004 

2023-05-08 10:34:00,373	WARNING util.py:244 -- The `start_trial` operation took 1.895 s, which may be a performance bottleneck.
2023-05-08 10:34:05,341	ERROR worker.py:408 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::ImplicitFunc.train() (pid=27716, ip=127.0.0.1, repr=train_model)
  File "/Users/alexisfrankson/opt/anaconda3/lib/python3.9/site-packages/ray/tune/trainable/trainable.py", line 384, in train
    raise skipped from exception_cause(skipped)
  File "/Users/alexisfrankson/opt/anaconda3/lib/python3.9/site-packages/ray/tune/trainable/function_trainable.py", line 336, in entrypoint
    return self._trainable_func(
  File "/Users/alexisfrankson/opt/anaconda3/lib/python3.9/site-packages/ray/tune/trainable/function_trainable.py", line 653, in _trainable_func
    output = fn()
  File "/var/folders/9f/cw5rv8456ld3sk0fl0q1xwjr0000gn/T/ipykernel_22594/1346344073.py", line 28, in train_model
  File "/Users/alexisfrankson/opt/anaconda3/lib/python3.9/site-package

== Status ==
Current time: 2023-05-08 10:34:00 (running for 00:00:39.15)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Logical resource usage: 2.0/4 CPUs, 0/0 GPUs
Result logdir: /Users/alexisfrankson/ray_results/train_model_2023-05-08_10-33-21
Number of trials: 10/10 (8 PENDING, 2 RUNNING)
+-------------------------+----------+-----------------+---------------+-------------+
| Trial name              | status   | loc             |   hidden_size |          lr |
|-------------------------+----------+-----------------+---------------+-------------|
| train_model_480b9_00000 | RUNNING  | 127.0.0.1:27716 |           500 | 0.0073379   |
| train_model_480b9_00001 | RUNNING  | 127.0.0.1:27724 |           500 | 0.00107933  |
| train_model_480b9_00002 | PENDING  |                 |           250 | 0.000305067 |
| train_model_480b9_00003 | PENDING  |                 |           250 | 0.00143448  |
| train_model_480b9_00004 

2023-05-08 10:34:08,295	WARNING util.py:244 -- The `start_trial` operation took 1.953 s, which may be a performance bottleneck.
2023-05-08 10:34:13,353	ERROR worker.py:408 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::ImplicitFunc.train() (pid=27724, ip=127.0.0.1, repr=train_model)
  File "/Users/alexisfrankson/opt/anaconda3/lib/python3.9/site-packages/ray/tune/trainable/trainable.py", line 384, in train
    raise skipped from exception_cause(skipped)
  File "/Users/alexisfrankson/opt/anaconda3/lib/python3.9/site-packages/ray/tune/trainable/function_trainable.py", line 336, in entrypoint
    return self._trainable_func(
  File "/Users/alexisfrankson/opt/anaconda3/lib/python3.9/site-packages/ray/tune/trainable/function_trainable.py", line 653, in _trainable_func
    output = fn()
  File "/var/folders/9f/cw5rv8456ld3sk0fl0q1xwjr0000gn/T/ipykernel_22594/1346344073.py", line 28, in train_model
  File "/Users/alexisfrankson/opt/anaconda3/lib/python3.9/site-package

...

2023-05-08 10:34:39,946	WARNING util.py:244 -- The `start_trial` operation took 1.866 s, which may be a performance bottleneck.
2023-05-08 10:34:43,393	ERROR worker.py:408 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::ImplicitFunc.train() (pid=27733, ip=127.0.0.1, repr=train_model)
  File "/Users/alexisfrankson/opt/anaconda3/lib/python3.9/site-packages/ray/tune/trainable/trainable.py", line 384, in train
    raise skipped from exception_cause(skipped)
  File "/Users/alexisfrankson/opt/anaconda3/lib/python3.9/site-packages/ray/tune/trainable/function_trainable.py", line 336, in entrypoint
    return self._trainable_func(
  File "/Users/alexisfrankson/opt/anaconda3/lib/python3.9/site-packages/ray/tune/trainable/function_trainable.py", line 653, in _trainable_func
    output = fn()
  File "/var/folders/9f/cw5rv8456ld3sk0fl0q1xwjr0000gn/T/ipykernel_22594/1346344073.py", line 28, in train_model
  File "/Users/alexisfrankson/opt/anaconda3/lib/python3.9/site-package

== Status ==
Current time: 2023-05-08 10:34:39 (running for 00:01:18.72)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Logical resource usage: 3.0/4 CPUs, 0/0 GPUs
Result logdir: /Users/alexisfrankson/ray_results/train_model_2023-05-08_10-33-21
Number of trials: 10/10 (4 ERROR, 3 PENDING, 3 RUNNING)
+-------------------------+----------+-----------------+---------------+-------------+
| Trial name              | status   | loc             |   hidden_size |          lr |
|-------------------------+----------+-----------------+---------------+-------------|
| train_model_480b9_00004 | RUNNING  | 127.0.0.1:27733 |           750 | 0.000160031 |
| train_model_480b9_00005 | RUNNING  | 127.0.0.1:27735 |           250 | 0.0117232   |
| train_model_480b9_00006 | RUNNING  | 127.0.0.1:27743 |           500 | 0.0409975   |
| train_model_480b9_00007 | PENDING  |                 |          1000 | 0.0134019   |
| train_model_480

2023-05-08 10:34:48,210	WARNING util.py:244 -- The `start_trial` operation took 1.900 s, which may be a performance bottleneck.
2023-05-08 10:34:52,407	ERROR worker.py:408 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::ImplicitFunc.train() (pid=27743, ip=127.0.0.1, repr=train_model)
  File "/Users/alexisfrankson/opt/anaconda3/lib/python3.9/site-packages/ray/tune/trainable/trainable.py", line 384, in train
    raise skipped from exception_cause(skipped)
  File "/Users/alexisfrankson/opt/anaconda3/lib/python3.9/site-packages/ray/tune/trainable/function_trainable.py", line 336, in entrypoint
    return self._trainable_func(
  File "/Users/alexisfrankson/opt/anaconda3/lib/python3.9/site-packages/ray/tune/trainable/function_trainable.py", line 653, in _trainable_func
    output = fn()
  File "/var/folders/9f/cw5rv8456ld3sk0fl0q1xwjr0000gn/T/ipykernel_22594/1346344073.py", line 28, in train_model
  File "/Users/alexisfrankson/opt/anaconda3/lib/python3.9/site-package

== Status ==
Current time: 2023-05-08 10:35:09 (running for 00:01:48.19)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Logical resource usage: 0/4 CPUs, 0/0 GPUs
Result logdir: /Users/alexisfrankson/ray_results/train_model_2023-05-08_10-33-21
Number of trials: 10/10 (10 ERROR)
+-------------------------+----------+-----------------+---------------+-------------+
| Trial name              | status   | loc             |   hidden_size |          lr |
|-------------------------+----------+-----------------+---------------+-------------|
| train_model_480b9_00000 | ERROR    | 127.0.0.1:27716 |           500 | 0.0073379   |
| train_model_480b9_00001 | ERROR    | 127.0.0.1:27724 |           500 | 0.00107933  |
| train_model_480b9_00002 | ERROR    | 127.0.0.1:27726 |           250 | 0.000305067 |
| train_model_480b9_00003 | ERROR    | 127.0.0.1:27727 |           250 | 0.00143448  |
| train_model_480b9_00004 | ERROR    | 1

TuneError: ('Trials did not complete', [train_model_480b9_00000, train_model_480b9_00001, train_model_480b9_00002, train_model_480b9_00003, train_model_480b9_00004, train_model_480b9_00005, train_model_480b9_00006, train_model_480b9_00007, train_model_480b9_00008, train_model_480b9_00009])